In [ ]:
import geopandas as gpd
import numpy as np
import folium
import rasterio
from functools import reduce
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import rasterio
# to calculate ndvi locally 
#import earthpy.spatial as es
import os

## Comparing bounding boxes and arrange the before/after images
We used this code to compare if two geotiffs belong to the same region by compatring their bounding box coordinates.
Later each was moved to a new seperate directories and renamed given the same name so we can later perform temporal analysis on them.

In [ ]:
# Set the paths to our data dir
data_before_dir = 'D:/dl_data/data224/Kenya_AOI_before_224'
data_after_dir = 'D:/dl_data/data224/Kenya_AOI_after_224'

new_before = 'D:/dl_data/data224/Kenya_AOI_before_224_arr'
new_after = 'D:/dl_data/data224/Kenya_AOI_after_224_arr'

In [ ]:
# How many images we got
img_before = []
img_after = []
img_dict = {}
for filename in os.listdir(data_before_dir):
    img_before.append(filename)

for filename in os.listdir(data_after_dir):
    img_after.append(filename)
    
print(f"We have {len(img_before)} before tha ttack || We have {len(img_after)} after the attack")

#we need to find which are belonging to the same region
# by checch=king the bonds
for filename_before in img_before:
    for filename_after in img_after:
        # Check if the mgrs from geojson is belongs to one of the images
        ratser_before = rasterio.open(data_before_dir +'/' + filename_before)
        ratser_after = rasterio.open(data_after_dir + '/'+ filename_after)
    
        if ratser_before.bounds == ratser_after.bounds:
            # pair the two
            img_dict[filename_before] = filename_after
            ratser_before.close()
            ratser_after.close()
           

    #Move the corresponding files to the new dir and give them the same name
for index, key in enumerate(img_dict):
    file_before = key
    file_after = img_dict[key]
    os.rename(data_before_dir + '/' + file_before, new_dir_before + '/' + file_before)
    os.rename(data_after_dir + '/' + file_after, new_dir_after + '/' + file_before)

## Crop Geotiffs

In [ ]:
data_before_dir = 'D:/dl_data/Kenya_AOI_before_224'

resized_before_dir = 'D:/dl_data/kenya_before_224_resized'

In [ ]:
import rasterio
from rasterio.windows import Window
# Get the number of imges within each folder
before_imgs = []
geotif_size = 224
for filename in os.listdir(data_before_dir):
    before_imgs.append(filename)

#Resize the images in our directories
for filename in before_imgs:
    with rasterio.open(data_before_dir + '/' + filename) as src:
        window = Window(0, 0,geotif_size, geotif_size)

        kwargs = src.meta.copy()
        kwargs.update({
            'height': window.height,
            'width': window.width,
            'transform': rasterio.windows.transform(window, src.transform)})

        with rasterio.open(resized_before_dir + '/' + filename, 'w', **kwargs) as dst:
            dst.write(src.read(window=window))
    